In [1]:
!pip install txtai[all] > /dev/null

Workflows are a simple yet powerful construct that takes a callable and returns elements. Workflows operate well with pipelines but can work with any callable object. Workflows are streaming and work on data in batches, allowing large volumes of data to be processed efficiently.

Workflows combine machine-learning pipelines together to aggregate logic. This application provides a number of pre-configured workflows to get a feel of how they work. Workflows can be exported and run locally through FastAPI. Read more on GitHub and in the Docs.

In [2]:
mult_2 = lambda x: [y * 2 for y in x]

In [4]:
from txtai.workflow import FileTask, Task, Workflow

workflow = Workflow([Task(mult_2)])
list(workflow([1, 2, 3]))

[2, 4, 6]

In [6]:
from txtai.pipeline import Summary

summary = Summary()
new_task = Task(summary)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
workflow = Workflow([new_task])

list(workflow(["Very long text here"]))

['Very long text here']

In [9]:
workflow = Workflow([Task([lambda x: [y * 3 for y in x], 
                           lambda x: [y - 1 for y in x]],
                           unpack=False, column={0:0, 1:1})])
list(workflow([(2, 8),(4, 18)]))

[(6, 7), (12, 17)]

In [10]:
workflow = Workflow([Task([lambda x: [y * 3 for y in x], 
                           lambda x: [y - 1 for y in x]],
                           unpack=True, column={0:0, 1:1})])
list(workflow([(2, 8),(4, 18)]))

[(2, (24, 7)), (4, (54, 17))]

The Console Task prints task inputs and outputs to standard output. This task is mainly used for debugging and can be added at any point in a workflow.

In [11]:
from txtai.workflow import FileTask, Workflow, ConsoleTask

workflow = Workflow([ConsoleTask()])
workflow(["Input 1", "Input2"])

<generator object Workflow.__call__ at 0x7fb02c5f02e0>